<a href="https://colab.research.google.com/github/alisonnnnn88/programming_language/blob/main/HW1_%E6%8F%90%E6%AC%BE%E6%A9%9F%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ==== 歡迎使用提款機系統 ====
📌 提款機系統使用說明
  
> 未使用過此系統之使用者，請點選【註冊】   
> 使用過則點選【登入】    
> 登入後，會有以下幾個鈕:  
> 輸入收入、輸入支出、查看餘額、清除紀錄、登出
>   
> 溫馨提醒:  
> 清除紀錄後 ⚠️不可復原⚠️   
> 支出時若金額超過餘額，會顯示「餘額不足」  


💵 提款紀錄: https://docs.google.com/spreadsheets/d/1h75idXVO6GMosL5bPUF-G76MqxOziHu1tJieRtpdkP4/edit?usp=sharing


In [24]:
# 安裝必要套件
!pip install -q gspread gspread_dataframe ipywidgets

# 匯入模組
import gspread
from google.colab import auth
from google.auth import default
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# Google 認證
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# ======= 設定你的 Google Sheet URL =======
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1h75idXVO6GMosL5bPUF-G76MqxOziHu1tJieRtpdkP4/edit?usp=sharing"
sh = gc.open_by_url(SPREADSHEET_URL)

# ======= 工作表設定 =======
try:
    user_ws = sh.worksheet("使用者資料表")
except gspread.WorksheetNotFound:
    user_ws = sh.add_worksheet(title="使用者資料表", rows="100", cols="2")
    user_ws.append_row(["使用者名稱", "密碼"])

try:
    record_ws = sh.worksheet("交易紀錄表")
except gspread.WorksheetNotFound:
    record_ws = sh.add_worksheet(title="交易紀錄表", rows="1000", cols="6")
    record_ws.append_row(["使用者名稱", "日期", "時間", "收入", "支出", "餘額"])

current_user = None

# ======= 共用元件 =======
username_input = widgets.Text(description="帳號：")
password_input = widgets.Password(description="密碼：")

# ======= 主選單 =======
def show_main_menu():
    clear_output()
    login_btn = widgets.Button(description="登入", button_style="primary")
    register_btn = widgets.Button(description="註冊", button_style="info")
    login_btn.on_click(handle_login)
    register_btn.on_click(handle_register)
    print("請選擇操作：")
    display(login_btn, register_btn)

# ======= 註冊 =======
def handle_register(_):
    clear_output()
    register_submit = widgets.Button(description="送出", button_style="success")
    cancel_btn = widgets.Button(description="取消", button_style="danger")
    display(username_input, password_input, register_submit, cancel_btn)

    def process_register(b):
        username = username_input.value.strip()
        password = password_input.value.strip()
        users = [str(u).strip() for u in user_ws.col_values(1) if u is not None]

        if not username or not password:
            print("❌ 請輸入帳號與密碼")
        elif username in users:
            print("⚠️ 帳號已存在")
        else:
            user_ws.append_row([username, password])
            clear_output()
            print(f"✅ 註冊成功！請回主選單按登入鍵登入")
            show_main_menu()

    register_submit.on_click(process_register)

    def cancel_register(b):
        clear_output()
        show_main_menu()
    cancel_btn.on_click(cancel_register)

# ======= 登入 =======
def handle_login(_):
    clear_output()
    login_submit = widgets.Button(description="送出", button_style="success")
    cancel_btn = widgets.Button(description="取消", button_style="danger")
    display(username_input, password_input, login_submit, cancel_btn)

    def process_login(b):
        global current_user
        username = username_input.value.strip()
        password = password_input.value.strip()

        users_dict = {str(row['使用者名稱']).strip(): str(row['密碼']).strip()
                      for row in user_ws.get_all_records() if row['使用者名稱'] is not None}

        if username in users_dict and users_dict[username] == password:
            current_user = username
            clear_output()
            print(f"✅ 登入成功，歡迎 {username}")
            enter_btn = widgets.Button(description="👉 進入提款機系統", button_style="success")
            enter_btn.on_click(lambda b: atm(current_user))
            display(enter_btn)
        else:
            print("❌ 帳號或密碼錯誤")

    login_submit.on_click(process_login)

    def cancel_login(b):
        clear_output()
        show_main_menu()
    cancel_btn.on_click(cancel_login)

# ======= 提款機功能 =======
def atm(user):
    clear_output()
    print(f"👤 使用者：{user}")
    print("請選擇功能：")

    btn_income = widgets.Button(description="1️⃣ 輸入收入", button_style="success")
    btn_expense = widgets.Button(description="2️⃣ 輸入支出", button_style="warning")
    btn_balance = widgets.Button(description="3️⃣ 查看餘額", button_style="info")
    btn_clear = widgets.Button(description="4️⃣ 清除紀錄", button_style="danger")
    btn_logout = widgets.Button(description="5️⃣ 登出", button_style="")

    # ======= 新增交易紀錄 =======
    def add_record(income=0, expense=0):
        now = datetime.now()
        df = pd.DataFrame(record_ws.get_all_records())
        if not df.empty:
            df.columns = df.columns.str.strip()
        user_df = df[df["使用者名稱"] == user] if not df.empty else pd.DataFrame()
        previous_balance = float(user_df.iloc[-1]["餘額"]) if not user_df.empty else 0.0
        new_balance = previous_balance + float(income) - float(expense)
        record_ws.append_row([user, now.strftime("%Y-%m-%d"), now.strftime("%H:%M:%S"),
                              float(income), float(expense), new_balance])

    # ======= 收入 =======
    def handle_income(b):
        clear_output()
        income_input = widgets.FloatText(description="收入金額：")
        confirm_btn = widgets.Button(description="確認", button_style="success")
        cancel_btn = widgets.Button(description="取消", button_style="danger")
        display(income_input, confirm_btn, cancel_btn)

        def confirm_income(btn):
            add_record(income=income_input.value)
            clear_output()
            print(f"✅ 已新增收入：{income_input.value}")
            atm(user)
        confirm_btn.on_click(confirm_income)

        def cancel_income(btn):
            clear_output()
            atm(user)
        cancel_btn.on_click(cancel_income)

    # ======= 支出 =======
    def handle_expense(b):
        clear_output()
        expense_input = widgets.FloatText(description="支出金額：")
        confirm_btn = widgets.Button(description="確認", button_style="success")
        cancel_btn = widgets.Button(description="取消", button_style="danger")
        display(expense_input, confirm_btn, cancel_btn)

        def confirm_expense(btn):
            amount = float(expense_input.value)
            df = pd.DataFrame(record_ws.get_all_records())
            if not df.empty:
                df.columns = df.columns.str.strip()
            user_df = df[df["使用者名稱"] == user] if not df.empty else pd.DataFrame()
            balance = float(user_df.iloc[-1]["餘額"]) if not user_df.empty else 0.0
            if amount > balance:
                clear_output()
                print("❌ 餘額不足！")
                return_btn = widgets.Button(description="返回選單", button_style="warning")
                def back(btn):
                    clear_output()
                    atm(user)
                return_btn.on_click(back)
                display(return_btn)
            else:
                add_record(expense=amount)
                clear_output()
                print(f"✅ 已新增支出：{amount}")
                atm(user)
        confirm_btn.on_click(confirm_expense)

        def cancel_expense(btn):
            clear_output()
            atm(user)
        cancel_btn.on_click(cancel_expense)

    # ======= 查看餘額 =======
    def handle_balance(b):
        clear_output()
        df = pd.DataFrame(record_ws.get_all_records())
        if not df.empty:
            df.columns = df.columns.str.strip()
        user_df = df[df["使用者名稱"] == user] if not df.empty else pd.DataFrame()
        balance = float(user_df.iloc[-1]["餘額"]) if not user_df.empty else 0.0
        print(f"💰 您目前的餘額為：{balance:.2f} 元")
        return_btn = widgets.Button(description="返回選單", button_style="info")
        def back(btn):
            clear_output()
            atm(user)
        return_btn.on_click(back)
        display(return_btn)

    # ======= 清除紀錄 =======
    def handle_clear(b):
        df = pd.DataFrame(record_ws.get_all_records())
        if not df.empty:
            df.columns = df.columns.str.strip()
        new_df = df[df["使用者名稱"] != user] if not df.empty else pd.DataFrame()
        record_ws.clear()
        record_ws.append_row(["使用者名稱", "日期", "時間", "收入", "支出", "餘額"])
        for row in new_df.values.tolist():
            record_ws.append_row(row)
        clear_output()
        print("🗑️ 已清除您的所有紀錄")
        return_btn = widgets.Button(description="返回選單", button_style="info")
        def back(btn):
            clear_output()
            atm(user)
        return_btn.on_click(back)
        display(return_btn)

    # ======= 登出 =======
    def handle_logout(b):
        global current_user
        current_user = None
        clear_output()
        print("👋 已登出")
        show_main_menu()

    btn_income.on_click(handle_income)
    btn_expense.on_click(handle_expense)
    btn_balance.on_click(handle_balance)
    btn_clear.on_click(handle_clear)
    btn_logout.on_click(handle_logout)

    display(btn_income, btn_expense, btn_balance, btn_clear, btn_logout)

# ======= 啟動主選單 =======
show_main_menu()

請選擇操作：


Button(button_style='primary', description='登入', style=ButtonStyle())

Button(button_style='info', description='註冊', style=ButtonStyle())